In [1]:
import torch
import torch.nn as nn

In [2]:
torch.cuda.is_available()

True

# Patch embedding

In [3]:
# B, C, H, W
img = torch.rand((100, 3, 224, 224))
print(img.shape)

torch.Size([100, 3, 224, 224])


In [5]:
img_size = (224, 224)
patch_size = (16, 16)
grid_size = (img_size[0]//patch_size[0], img_size[1]//patch_size[1])
num_patches = grid_size[0] * grid_size[1]
proj = nn.Conv2d(3, 768, kernel_size=patch_size, stride=patch_size)
norm = nn.Identity()

In [6]:
B, C, H, W = img.shape
x = proj(img)
print("[Batch, embed_dim, H_Patch, W_patch]")
print(x.shape)
x = x.flatten(2)
print("[Batch, embed_dim, num_patch]")
print(x.shape)
x = x.transpose(1,2)
print("[Batch, Patch, embed_dim]")
print(x.shape)

[Batch, embed_dim, H_Patch, W_patch]
torch.Size([100, 768, 14, 14])
[Batch, embed_dim, num_patch]
torch.Size([100, 768, 196])
[Batch, Patch, embed_dim]
torch.Size([100, 196, 768])


# Attention

In [22]:
num_heads = 16
dim = 768
head_dim = dim//num_heads
qkv = nn.Linear(dim, dim*3, bias = False)
scale = head_dim ** -0.5
attn_drop = 0.2
proj1 = nn.Linear(dim, dim)
proj_drop = nn.Dropout(0.2)
proj2 = nn.Linear(dim, dim)

In [29]:
cls_token = nn.Parameter(torch.zeros(1, 1, 768))
cls_token = cls_token.expand(x.shape[0], -1, -1)
print(cls_token.shape)

torch.Size([100, 1, 768])


In [30]:
x = torch.cat((cls_token, x), dim=1)
print(x.shape)

torch.Size([100, 197, 768])


In [31]:
B, N, C = x.shape
print("[B, num_patches+1, embed_dim]")
print(x.shape)
x = qkv(x)
print("[B, num_patchs+1, qkv_dim]")
print(x.shape)
x = x.reshape(B, N, 3, num_heads, C//num_heads)
print("[B, num_patch+1, qkv, num_heads, heads_dim]")
print(x.shape)
x = x.permute(2, 0, 3, 1, 4)
print("[qkv, batch_size, num_heads, num_patches + 1, embed_dim_per_head]")
print(x.shape)

[B, num_patches+1, embed_dim]
torch.Size([100, 197, 768])
[B, num_patchs+1, qkv_dim]
torch.Size([100, 197, 2304])
[B, num_patch+1, qkv, num_heads, heads_dim]
torch.Size([100, 197, 3, 16, 48])
[3, batch_size, num_heads, num_patches + 1, embed_dim_per_head]
torch.Size([3, 100, 16, 197, 48])


In [34]:
q, k, v = x[0], x[1], x[2]
print("q: ", q.shape)
print("k: ", k.shape)
print("v: ", v.shape)

q:  torch.Size([100, 16, 197, 48])
k:  torch.Size([100, 16, 197, 48])
v:  torch.Size([100, 16, 197, 48])


In [36]:
attn = (q @ k.transpose(-2, -1))
attn = attn.softmax(dim=-1)
print(attn.shape)

torch.Size([100, 16, 197, 197])


In [38]:
x = (attn @ v).transpose(1, 2).reshape(B, N, C)
x = proj2(x)
print(x.shape)

torch.Size([100, 197, 768])


# MyCode

In [4]:
import torch
from vit_model import Attention
from vit_model import PathEmbed
from vit_model import Attention
from vit_model import MLP

In [8]:
img = torch.rand((100, 3, 224, 224))
embeddingLayer = PathEmbed()
x = embeddingLayer(img)
print(x.shape)

torch.Size([100, 196, 768])


In [10]:
multiHeadAttention = Attention(dim=768, num_heads=16)
x = multiHeadAttention(x)
print(x.shape)

torch.Size([100, 196, 768])


In [12]:
mlp = MLP(in_features=768)
x = mlp(x)
print(x.shape)

torch.Size([100, 196, 768])


In [1]:
from vit_model import vit_base_patch16_224_in21k

In [2]:
model = vit_base_patch16_224_in21k(has_logits=False)

In [3]:
img = torch.rand((100, 3, 224, 224))
X = model.forward(img)

SyntaxError: invalid syntax (2821267061.py, line 2)